In [ ]:
import os
import pandas as pd
import re
from itertools import combinations
import json
import numpy as np
import time

In [ ]:
directory = "columns\\"

In [ ]:
def jaccard(set1, set2):
    if len(set1) > 0 or len(set2) > 0:
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        return len(intersection) / len(union)
    return 0

In [ ]:
threshold = 0.2

directory = "columns"
name = "finalschema"
final_schema_directory = "dictionary"

result_jaccard = {}
for filename in os.listdir(directory):

    print(filename)
    file = os.path.join(directory, filename)
    print(file)
    df = pd.read_csv(file, encoding="utf-8", low_memory=False)
    print("Numero colonne: ", len(list(df.columns)))
    
    start = time.time()
    combs = list(combinations(list(df.columns), 2))
    end = time.time()
    
    print("Numbero di confronti: ", len(combs), "-", str(end-start))
    k = 0
    for c1, c2 in combs:
        
        start = time.time()
        s = jaccard(set(list(df[c1])), set(list(df[c2])))
        end = time.time()
        
        print(str(k), "-", c1, "-", c2, "-", str(end-start))
        k += 1

        if s > threshold:

            if c1 not in result_jaccard.keys():
                result_jaccard[c1] = set()
            if c2 not in result_jaccard.keys():
                result_jaccard[c2] = set()

            result_jaccard[c1].add(c2)
            result_jaccard[c2].add(c1)
    
    print("\n")

s = set()
for main_token, tokens in result_jaccard.items():
    tmp = set()
    tmp.add(main_token)
    tmp.update(tokens)
    s.add(frozenset(tmp))

i = 0
dict_synonyms = {}
for tokens in s:
    tokens = list(tokens)
    tmp = tokens[0].split("-")[2]
    if tmp not in dict_synonyms.keys():
        dict_synonyms[tmp] = tokens
    else:
        dict_synonyms[tmp + "-" + str(i)] = tokens
        i += 1

json_obj = json.dumps(dict_synonyms, indent=4)
f = open(final_schema_directory + name + ".json", "w")
f.write(json_obj)
f.close()

print("\n\n")

In [ ]:
import numpy as np
from Levenshtein import distance

In [ ]:
def my_parser(s):
    if s is not np.nan:
        s = str(s)
        p = re.compile(r'\d+\.\d+')
        tmp = [float(i) for i in p.findall(s)]
        return tmp[0] if len(tmp) > 0 else 0
    else:
        return 0

In [ ]:
def my_parser2(s):
    if s is not np.nan:
        return str(s)
    return ""

In [ ]:
threshold_mean = 1000
threshold_var = 1000
threshold = 5

cluster_distribution = {}

for dirname in os.listdir(directory):
    
    print(dirname)
    cluster = os.path.join(directory, dirname)
    
    result_distribution = {}
    for filename in os.listdir(cluster):

        print(filename)
        file = os.path.join(cluster, filename)
        df = pd.read_csv(file, encoding="utf-8")
        for c1, c2 in list(combinations(list(df.columns), 2)):

            column1 = list(df[c1].apply(my_parser))
            column2 = list(df[c2].apply(my_parser))
            
            column1 = sorted(column1, reverse=True)
            column2 = sorted(column2, reverse=True)
            
            column1 = list(map(str, column1))
            column2 = list(map(str, column2))
            
            #mean1 = np.mean(column1)
            #mean2 = np.mean(column2)
            
            #var1 = np.var(column1)
            #var2 = np.var(column2)
            
            #s1 = abs(mean1-mean2)
            #s2 = abs(var1-var2)
            
            s = sum(distance(s1, s2) for s1, s2 in zip(column1, column2))
            s = s / max(len(column1), len(column2))
            
            if 0 < s < threshold:
                        
                print(c1, c2, s)

                if c1 not in result_distribution.keys():
                    result_distribution[c1] = set()
                if c2 not in result_distribution.keys():
                    result_distribution[c2] = set()

                result_distribution[c1].add(c2)
                result_distribution[c2].add(c1)

    s = set()
    for main_token, tokens in result_distribution.items():
        tmp = set()
        tmp.add(main_token)
        tmp.update(tokens)
        s.add(frozenset(tmp))

    i = 0
    dict_synonyms = {}
    for tokens in s:
        tokens = list(tokens)
        tmp = tokens[0].split("-")[2]
        if tmp not in dict_synonyms.keys():
            dict_synonyms[tmp] = tokens
        else:
            dict_synonyms[tmp + "-" + str(i)] = tokens
            i += 1

    json_obj = json.dumps(dict_synonyms, indent=4)
    f = open(dirname + ".json", "w")
    f.write(json_obj)
    f.close()

    print("\n\n")